In [ ]:
import pandas as pd
from pathlib import Path
from typing import Tuple, List

from owforecasting.features import default_features
from owforecasting import TimeSeries

from forecasting_platform import master_config

### Configure directory path to load:
- `"00 Config"`
- `"01 Raw data"`
- `"03 Processed data"`

In [ ]:
DATA_DIRECTORY = '../'
master_config.default_data_loader_location = Path(DATA_DIRECTORY)

In [ ]:
from forecasting_platform.helpers import AccountID, ProjectID, ContractID
from forecasting_platform.model_config_scripts import BaseModelConfig
from forecasting_platform.static import EngineRunType, OutputFormat, InternalFeatures, ExogenousFeatures, Weights
from forecasting_platform.cli.forecast import run_forecast
from forecasting_platform.forecasting.postprocess import postprocess_forecast_results

# Create new ModelConfig class based on `BaseModelConfig`
For inspiration, you can take a look at some of the existing accounts:
- Simple: [Account_2](https://bitbucket.org/oliverwymantechssg/pap007/src/master/forecasting_platform/model_config_scripts/FC_Account2.py)
- With additional pre-processing: [Account_8](https://bitbucket.org/oliverwymantechssg/pap007/src/master/forecasting_platform/model_config_scripts/FC_Account8.py)
- With airframes feature: [Account_3](https://bitbucket.org/oliverwymantechssg/pap007/src/master/forecasting_platform/model_config_scripts/FC_Account3.py)
- With custom post-processing: [Account_3](https://bitbucket.org/oliverwymantechssg/pap007/src/master/forecasting_platform/model_config_scripts/FC_Account3.py)

In [ ]:
class ModelConfigAccount99(BaseModelConfig):
    ## These are the normal parameters defined for each account model config.
    ## For normal cases, these should be sufficient to achieve reasonable forecasts.
    
    MODEL_NAME = AccountID("Account 99")

    CONTRACTS = [ContractID("Contract_601")]
    
    TRAINING_START = pd.Timestamp(year=2018, month=1, day=1)

    GROUPING = ["Contract_ID", "Item_ID"]

    POSTPROCESS_DEPTH = 4

    DEFAULT_FEATURES = default_features(
        lag=[1, 3, 5],
        moving_window=[2, 3],
    )

    WEIGHTING = 5
    
    ## Following are optional parameters, only used by some accounts
    ## (The defaults here have no effect and just document their usage)
    ## Please keep in mind to reuse existing parameters and functions to keep complexity manageable and reduce project risks.
    
    EXCLUDE_PROJECTS = [
        # Here you can specify a list of projects to exclude from sales during pre-processing, e.g.:
        # ProjectID("Project_1574"),
    ]
    
    
    ONLY_INCLUDE_PROJECTS = [
        # Here you can specify a list of projects to exclusively include in sales during pre-processing, e.g.:
        # ProjectID("Project_1574"),
    ]
    
    
    EXCLUDE_ITEMS = [
        # Here you can specify a list of items to exclude from sales during pre-processing, e.g.:
        # 923853, 285331
    ]
    
            
    ONLY_INCLUDE_ITEMS = [
        # Here you can specify a list of items to exclusively include in sales during pre-processing, e.g.:
        # 13424, 234555
    ]
    
    
    # Additional pre-processing of outliers on the sales training data can be enabled, 
    PREPROCESS_OUTLIERS = False  # In most cases, it should not be necessary to set this to `True`
    

    # Aggregation of "Unit_Cost" can be adjusted
    PREPROCESS_UNIT_COST_AGGREGATION = "mean" # Can be set to other Pandas aggregation operators like `max`

    
    # Number of months to filter sales during pre-processing
    SALES_MIN_PERIOD = 5
    
    
    OVERRIDE_HYPER_PARAMS = {
        # Custom hyper-parameters can be specified, these will be merged with the defaults of the owforecasting package.
        # Existing default parameters will be replaced.
        # e.g. "nfolds": 5,
    }
    
    # Optionally change the `n_calls` parameter of the `optimize_bayes` function. Default: 20
    # OPTIMIZE_HYPER_PARAMETERS_N_CALLS = 10
    
    # HYPER_SPACE = [
        # Here you can adjust specify hyper-space dimensions to use during hyper-parameter optimization.
        # Default is specified by the `get_default_hyper_space` method of the owforecasting package
    # ]

    
    ##### Following are optional methods, that can be overridden to hook into the forecasting pipeline
    ##### It is important to keep the existing function signature (parameters and return types).
    ##### (The defaults here have no effect and can also be omitted)
    
    def configure_features(self, cleaned_data_run_id: int) -> Tuple[InternalFeatures, ExogenousFeatures]:
        # Can be used to implement custom features like build-rates or airframes.
        # For example usage, see https://bitbucket.org/oliverwymantechssg/pap007/src/master/forecasting_platform/model_config_scripts/FC_Account3.py
        return {}, {}
    
    # def calculate_weights(self) -> Weights:
        # Can be used to configure account-specific weighting, e.g. weighting based on market shock event."""
        # For example usage, see https://bitbucket.org/oliverwymantechssg/pap007/src/master/forecasting_platform/model_config_scripts/FC_Account2.py
    #    return super().calculate_weights()
    
    def preprocess_account_data(
           self, sales_raw: pd.DataFrame, grouping: List[str], internal_features: InternalFeatures
       ) -> pd.DataFrame:
        # You can hook into the preprocessing step, before or after the account-agnostic steps are performed.
        # e.g. account-specific filtering or mapping of sales data
        # For example usage, see https://bitbucket.org/oliverwymantechssg/pap007/src/master/forecasting_platform/model_config_scripts/FC_Account8.py
        return super().preprocess_account_data(sales_raw, grouping, internal_features)

    def prepare_training_data(
            self, sales: pd.DataFrame, grouping: List[str], exo_features: ExogenousFeatures,
        ) -> TimeSeries:
        # You can hook into the feature engineering step to customize training data, before the model is trained.
        # For example usage, see https://bitbucket.org/oliverwymantechssg/pap007/src/master/forecasting_platform/model_config_scripts/FC_Account6.py
        return super().prepare_training_data(sales, grouping, exo_features)

    def postprocess_forecast(self, ts: TimeSeries, ts_pred: TimeSeries, sales: pd.DataFrame, grouping: List[str]) -> pd.DataFrame:
        # You can extend or replace the postprocessing step.
        # For example usage, see https://bitbucket.org/oliverwymantechssg/pap007/src/master/forecasting_platform/model_config_scripts/FC_Account3.py
        return postprocess_forecast_results(
            ts_pred.result_data,
            grouping,
            self._runtime_config.forecast_start,
            self.POSTPROCESS_DEPTH
        )
    

    # PLEASE keep in mind to reuse existing parameters and functions to keep complexity manageable and reduce project risks.
    

## Configure `master_config` for this specific execution
### Configure ModelConfig class to be executed

In [ ]:
only_model_config = ModelConfigAccount99

### Configure H2O Server connection

In [ ]:
master_config.h2o_urls = ["http://localhost:55555"]

### Disable multi-processing to allow changing `master_config` from this notebook

In [ ]:
master_config.max_parallel_models = 1

### Disable database connection

In [ ]:
master_config.db_connection_attempts = 0

### Configure directory path for output:
- `"08 Predictions"`

In [ ]:
OUTPUT_DIRECTORY = '../'

### Configure output format, choose:
- `OutputFormat.csv`
- `OutputFormat.xlsx`

In [ ]:
OUTPUT_FORMAT = OutputFormat.csv

### Configure run type:
- `EngineRunType.development` for forward run
- `EngineRunType.backward` for backward validation run

In [ ]:
RUN_TYPE = EngineRunType.backward

### Configure first month of forecast
- for backward forecast this is the last included month of the forecast period, for development/production forecast this is the first included month of the forecast period, if seen chronologically.

In [ ]:
PREDICTION_MONTH = pd.Timestamp(year=2020, month=2, day=1)

### Configure number of forecast months

In [ ]:
FORECAST_PERIODS = 1

## Optionally enable hyper-parameter optimization
Output will be logged and written to JSON file in the forecast directory within "08 Predictions".

In [ ]:
ENABLE_HYPER_PARAMETER_OPTIMIZATION = False

## Run forecast based on given parameters

In [ ]:
run_forecast(
    engine_run_type=RUN_TYPE,
    forecast_periods=FORECAST_PERIODS,
    output_location=OUTPUT_DIRECTORY,
    prediction_month=PREDICTION_MONTH,
    output_format=OUTPUT_FORMAT,
    only_model_config=only_model_config,
    optimize_hyperparameters=ENABLE_HYPER_PARAMETER_OPTIMIZATION,
)